In [1]:
# Import libraries
import numpy as np 
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Prepare Dataset
# load data

# relative path to npz files
path = 'Measurements/manual single top deformed at 20Hz'
file_name = 'output_batch_%d.npz'

def load_file(path, batch_id):
    """
    Loads in the memory a specific batch located in the given path.

    Returns a numpy NpzFile object with the image frames np array named
    as "frames" and the force measurements named as "frames".
    """
    combined_path = os.path.join(path,file_name %i)
    data = np.load(combined_path)
    return data

In [3]:
# Create RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_channels, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(input_channels, 9, kernel_size = 5, stride = 1, padding = 2)
        self.pool = nn.MaxPool2d(kernel_size = 4, stride = 4)
        self.conv2 = nn.Conv2d(9, 18, kernel_size = 5, stride = 1, padding = 2)
        
        # RNN
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.RNN(18 * 16 * 30, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Convolutional layers
        x = self.pool(nn.ELU()(self.conv1(x)))
        x = self.pool(nn.ELU()(self.conv2(x)))
        
        # Reshape for RNN
        # x = torch.reshape(x, (100, 10, 18 * 16 * 30))
        x = torch.reshape(x, (10, 10, 18 * 16 * 30))  # Reshape to (batch_size, seq_len, input_size)
        print("Size of x:", x.size())  # Print size of x
        
        # RNN
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
        #print("Size of h0:", h0.size())  # Print size of h0 - Debugging - Obsolete
        out, hn = self.rnn(x, h0)
        
        # Output layer
        out = self.fc(out) 
        # out = torch.reshape(out, (1000, 1))
        out = torch.reshape(out, (100, 1)) # go back to compare to labels
        return out


In [4]:
# Create RNN
input_channels = 3  # RGB channels
hidden_dim = 200  # hidden layer dimension
layer_dim = 2     # number of hidden layers
output_dim = 1   # output dimension

model = RNNModel(input_channels, hidden_dim, layer_dim, output_dim)

# Define your loss function
error = nn.MSELoss()

# Define your optimizer
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
# Training

file_size = 1000 # one file contains 1000 samples
num_epochs = 5
files_num = 2 # how many files to load
seq_dim = 100
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
batch_size = 100 # should be something that leaves zero reminder when it divides the file_size
for epoch in range(num_epochs):
    for i in range(files_num):
        data = load_file(path, i)
        
        for i in range(int(file_size/batch_size)):
            images = torch.from_numpy(data['frames'][i*batch_size:(i+1)*batch_size,:,:,:]).float()
            labels = torch.from_numpy(data['forces'][i*batch_size:(i+1)*batch_size,2]).float()        
            # print(images.shape)  # Add this line to check the shape of images - Debugging purposes
            images = images.permute(0, 3, 1, 2)
                
            # Clear gradients
            optimizer.zero_grad()
            
            # Forward propagation
            outputs = model(images)
            outputs = torch.squeeze(outputs)
            
            # Calculate loss
            loss = error(outputs, labels)
            
            # Backpropagation
            loss.backward()
            
            # Update parameters
            optimizer.step()
            
            count += 1
                
            # Store loss and iteration
            loss_list.append(loss.data) #* (forces_z_std^2)
            # Print Loss
            if count % 1 == 0: # for now print for every iteration
                print('Iteration: {}  Loss: {}'.format(count, loss.data.item())) # * (forces_z_std^2)

Size of x: torch.Size([10, 10, 8640])
Iteration: 1  Loss: 25.964763641357422
Size of x: torch.Size([10, 10, 8640])
Iteration: 2  Loss: 333.91070556640625
Size of x: torch.Size([10, 10, 8640])
Iteration: 3  Loss: 43.58935546875
Size of x: torch.Size([10, 10, 8640])
Iteration: 4  Loss: 35.99770736694336
Size of x: torch.Size([10, 10, 8640])
Iteration: 5  Loss: 132.87515258789062
Size of x: torch.Size([10, 10, 8640])
Iteration: 6  Loss: 99.73667907714844
Size of x: torch.Size([10, 10, 8640])
Iteration: 7  Loss: 32.80963134765625
Size of x: torch.Size([10, 10, 8640])
Iteration: 8  Loss: 4.310567855834961
Size of x: torch.Size([10, 10, 8640])
Iteration: 9  Loss: 28.79064178466797
Size of x: torch.Size([10, 10, 8640])
Iteration: 10  Loss: 51.25714874267578
Size of x: torch.Size([10, 10, 8640])
Iteration: 11  Loss: 42.23223114013672
Size of x: torch.Size([10, 10, 8640])
Iteration: 12  Loss: 17.619287490844727
Size of x: torch.Size([10, 10, 8640])
Iteration: 13  Loss: 2.630267381668091
Size of

In [6]:
print(loss_list)
print(np.sqrt(np.mean(loss_list[-10:])))

[tensor(25.9648), tensor(333.9107), tensor(43.5894), tensor(35.9977), tensor(132.8752), tensor(99.7367), tensor(32.8096), tensor(4.3106), tensor(28.7906), tensor(51.2571), tensor(42.2322), tensor(17.6193), tensor(2.6303), tensor(9.7687), tensor(20.3786), tensor(21.2190), tensor(15.5197), tensor(5.3190), tensor(3.5091), tensor(8.2257), tensor(13.1644), tensor(8.1403), tensor(7.3764), tensor(2.9513), tensor(1.7214), tensor(4.2944), tensor(6.9579), tensor(4.6932), tensor(2.9348), tensor(1.8695), tensor(2.0914), tensor(4.1838), tensor(4.0310), tensor(2.9305), tensor(1.9485), tensor(2.2301), tensor(3.0100), tensor(3.0686), tensor(2.4153), tensor(2.3927), tensor(2.5683), tensor(1.1433), tensor(2.0864), tensor(1.9448), tensor(0.8858), tensor(0.7484), tensor(0.8978), tensor(1.4718), tensor(1.4839), tensor(1.5690), tensor(1.2589), tensor(1.6117), tensor(1.3731), tensor(1.5148), tensor(1.5680), tensor(1.8031), tensor(1.4772), tensor(1.1701), tensor(1.7263), tensor(1.9249), tensor(2.2286), tensor

In [9]:
print(outputs) # Small sanity check

tensor([-2.1077, -1.0038, -0.7995, -1.4391, -0.8869, -1.0202, -0.5590, -0.3168,
        -0.5819, -0.3737, -0.7441, -0.3708, -0.4237, -0.7871, -0.4383, -0.2764,
        -0.3442, -0.2389, -0.3068, -0.0540, -0.4404, -0.5306, -0.4157, -0.7802,
        -0.4667, -0.2456, -0.2674, -0.1639, -0.2056, -0.0587, -0.8580, -0.9465,
        -1.1103, -2.0071, -1.8482, -2.4549, -2.5280, -1.6560, -2.4050, -2.3047,
        -2.1320, -2.1171, -2.5134, -2.7933, -2.7854, -2.1296, -2.1062, -1.0555,
        -0.9705, -0.7812, -1.1566, -0.8603, -0.8723, -1.1510, -0.9536, -0.6834,
        -0.4529, -0.3104, -0.6955, -0.5685, -0.8926, -0.5106, -0.4582, -1.0635,
        -0.3563, -0.2808, -0.3622, -0.2676, -0.2375, -0.0143, -0.6605, -0.5229,
        -0.5316, -1.1870, -1.2191, -1.2330, -1.8680, -2.4337, -1.9806, -2.3055,
        -2.2983, -2.4070, -2.4080, -2.8696, -2.7027, -2.9402, -2.1496, -2.0185,
        -0.8987, -1.0645, -1.2537, -1.0703, -0.8970, -1.3609, -1.0010, -1.0393,
        -0.6597, -0.4886, -0.5171, -0.54

In [10]:
print(labels.float())

tensor([-0.9837, -0.1637, -0.1279, -0.1309, -0.1346, -0.1297, -0.1344, -0.1294,
        -0.1343, -0.1282, -0.1322, -0.1240, -0.1298, -0.1313, -0.1317, -0.1270,
        -0.1292, -0.1361, -0.1341, -0.1336, -0.1390, -0.1233, -0.1278, -0.1282,
        -0.1346, -0.1320, -0.1334, -0.1283, -0.1324, -0.3513, -1.0552, -1.5855,
        -2.5075, -2.8629, -3.2809, -3.5987, -4.0110, -4.4015, -4.5454, -4.9288,
        -5.1063, -5.1722, -4.9448, -4.5689, -3.3009, -1.5864, -0.2401, -0.1187,
        -0.1239, -0.1288, -0.1313, -0.1257, -0.1342, -0.1297, -0.1307, -0.1230,
        -0.1293, -0.1233, -0.1307, -0.1248, -0.1389, -0.1326, -0.1363, -0.1292,
        -0.1340, -0.1309, -0.1326, -0.1288, -0.1315, -0.1237, -0.1371, -0.3839,
        -0.8983, -1.3171, -2.2785, -2.7973, -3.4132, -3.8083, -4.2586, -4.6140,
        -4.6368, -4.7419, -4.7785, -4.7576, -4.3578, -3.5074, -1.8832, -0.4341,
        -0.1187, -0.1313, -0.1282, -0.1306, -0.1325, -0.1369, -0.1345, -0.1265,
        -0.1310, -0.1315, -0.1313, -0.12

Progress Summary: 2 Conv&Pool layers, RNN with 2 hidden states of 200 size, 10 sequence length, 5 epochs: 1.042 RMSE. Best performance by a small margin, albeit on training set.